In [1]:
from counterfactuals.explainers import Cadex
from counterfactuals.visualization import compare
from data import GermanData
from tensorflow import keras

In [2]:
german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')
test_X = german_data.X_test.iloc[6]
cadex = Cadex(model)
cf = cadex.generate(test_X)

In [3]:
df = compare(test_X, cf.iloc[0])
df

,X,X',change
duration,0.783963,2.017929,1.233966
savings_< 100 DM,0.799590,2.006820,1.207230
installment_percent,0.906821,2.096033,1.189212
credit_history_existing credits paid back duly till now,-1.080751,0.052732,1.133483
purpose_car (new),-0.550392,0.637194,1.187587


In [4]:
test_X = german_data.input.iloc[6]
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(test_X)
compare(test_X, cf.iloc[0])

,X,X',change
account_status_< 0 DM,0,0.996974,0.996974
credit_history_no credits taken,0,0.429891,0.429891
purpose_car (new),0,0.883945,0.883945
job_unskilled - resident,0,0.810646,0.810646
duration,24,48.510236,24.510236


In [5]:
test_X = german_data.X_test.iloc[10]
cadex = Cadex(model, n_changed=8)
cf = cadex.generate(test_X)
compare(test_X, cf.iloc[0])

,X,X',change
phone_none,0.825029,0.942725,0.117696
purpose_others,8.888194,9.005891,0.117696
account_status_no checking account,-0.793302,-0.675605,0.117697
credit_history_critical account,-0.631349,-0.513653,0.117696
purpose_radio/television,-0.612022,-0.494325,0.117697
employment_4..7 years,-0.464549,-0.346853,0.117696
purpose_car (used),-0.335643,-0.217947,0.117696
account_status_>= 200 DM,-0.271708,-0.154012,0.117696


In [6]:
from counterfactuals.explainers import ECE

ece = ECE(5, columns=list(german_data.X_train.columns), bces=[Cadex(model, n_changed=i) for i in range(10, 20)], dist=2, h=8, lambda_=0.2, n_jobs=1)

ece.generate(german_data.X_test.iloc[0])


,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,0.059620,-0.053562,-0.958725,-0.760852,0.232038,-0.706372,0.406092,0.121059,0.150494,-0.271708,...,0.318308,0.825029,-0.825029,-0.186893,1.019027,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
1,0.066552,-0.040339,-0.952554,-0.760852,0.232038,-0.706372,0.421184,0.111412,0.147955,-0.271708,...,0.313437,0.825029,-0.825029,-0.186893,1.032058,-0.717741,-0.464549,0.180219,-0.592749,4.19579


In [7]:
ece.get_aggregated_cfs()

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,0.059620,-0.053562,-0.958725,-0.760852,0.232038,-0.706372,0.406092,0.121059,0.150494,-0.271708,...,0.318308,0.825029,-0.825029,-0.186893,1.019027,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
1,0.066552,-0.040339,-0.952554,-0.760852,0.232038,-0.706372,0.421184,0.111412,0.147955,-0.271708,...,0.313437,0.825029,-0.825029,-0.186893,1.032058,-0.717741,-0.464549,0.180219,-0.592749,4.19579
2,0.082944,-0.023858,-0.931508,-0.760852,0.232038,-0.706372,0.431841,0.122514,0.173270,-0.271708,...,0.339588,0.825029,-0.825029,-0.186893,1.046461,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
3,0.107495,-0.009574,-0.914969,-0.760852,0.232038,-0.706372,0.454475,0.193996,0.209631,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,1.066225,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
4,0.187840,0.068550,-0.826758,-0.760852,0.232038,-0.706372,0.555975,0.209693,0.265807,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,0.186893,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
5,0.267055,0.127366,-0.752003,-0.760852,0.232038,-0.706372,-0.430331,-0.613951,0.355589,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,0.186893,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
6,0.267530,0.129864,-0.750142,-0.760852,0.232038,-0.706372,-0.430331,0.224657,0.318226,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,0.186893,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
7,0.376424,0.248161,-0.657368,-0.760852,0.232038,-0.706372,-0.430331,-0.613951,0.438935,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,0.186893,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
8,0.464106,0.324443,-0.574537,-0.760852,0.232038,-0.706372,-0.430331,-0.613951,0.506843,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,0.186893,-0.717741,-0.464549,-0.450564,-0.592749,4.19579
9,0.527444,0.380768,-0.514037,-0.760852,0.232038,-0.706372,-0.430331,-0.613951,0.563522,-0.271708,...,-0.507801,0.825029,-0.825029,-0.186893,0.186893,-0.717741,-0.464549,-0.450564,-0.592749,4.19579


In [8]:
german_data.X_train.iloc[0]

duration                -0.742825
credit                  -0.433363
installment_percent      0.906821
residence_duration      -0.760852
age                      2.270963
                           ...   
employment_1..4 years   -0.717741
employment_4..7 years    2.152627
employment_< 1 year     -0.450564
employment_>= 7 years   -0.592749
employment_unemployed   -0.238334
Name: 96, Length: 61, dtype: float64